## Open Meteo API

In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [17]:
# from datetime import date, datetime, timedelta

# start_date = date(2024,1,1)
# end_date = date(2025,6,15)

# date_list = []
# current_date = start_date
# while current_date <= end_date:
#     date_list.append(current_date.strftime("%Y-%m-%d"))
#     current_date += timedelta(days=1)

In [28]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"

lat_long_list = [
    [-6.219151, 106.801624],
    [-6.219972, 106.802643],
    [-6.219679, 106.803190],
    [-6.219167, 106.803645],
    [-6.218602, 106.803914],
    [-6.218090, 106.803758],
    [-6.217338, 106.803271],
    [-6.216970, 106.802671],
    [-6.217263, 106.802112],
    [-6.217887, 106.801549],
    [-6.218538, 106.801346]
]

# Define column names
columns = ['date', 'temperature_2m', 'wind_speed_10m', 'wind_direction_10m']

# Store rows in a list
all_data = [] 

for lat_long in lat_long_list:
    params = {
        "latitude": lat_long[0],
        "longitude": lat_long[1],
        "hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m"],
        "timezone": "Asia/Bangkok",
        "start_date": "2024-01-01",
        "end_date": "2025-06-15"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Latitude {lat_long[0]} and Longitude {lat_long[1]}")
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
#     print(f"Elevation {response.Elevation()} m asl")
#     print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
#     print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["latitude"] = lat_long[0]
    hourly_data["longitude"] = lat_long[1]

    hourly_df = pd.DataFrame(hourly_data)
    hourly_df["date"] = hourly_df["date"].dt.tz_convert("Asia/Jakarta")
    hourly_df_filtered = hourly_df[hourly_df["date"].dt.hour.isin([20, 21])]

    all_data.append(hourly_df_filtered)  # Store filtered results
    

final_df = pd.concat(all_data, ignore_index=True)

Latitude -6.219151 and Longitude 106.801624
Coordinates -6.25°N 106.75°E
Latitude -6.219972 and Longitude 106.802643
Coordinates -6.25°N 106.75°E
Latitude -6.219679 and Longitude 106.80319
Coordinates -6.25°N 106.75°E
Latitude -6.219167 and Longitude 106.803645
Coordinates -6.25°N 106.75°E
Latitude -6.218602 and Longitude 106.803914
Coordinates -6.25°N 106.75°E
Latitude -6.21809 and Longitude 106.803758
Coordinates -6.25°N 106.75°E
Latitude -6.217338 and Longitude 106.803271
Coordinates -6.25°N 106.75°E
Latitude -6.21697 and Longitude 106.802671
Coordinates -6.25°N 106.75°E
Latitude -6.217263 and Longitude 106.802112
Coordinates -6.25°N 106.75°E
Latitude -6.217887 and Longitude 106.801549
Coordinates -6.25°N 106.75°E
Latitude -6.218538 and Longitude 106.801346
Coordinates -6.25°N 106.75°E


In [29]:
final_df

,date,temperature_2m,wind_speed_10m,wind_direction_10m,latitude,longitude
0,2024-01-01 20:00:00+07:00,NaN,NaN,NaN,-6.219151,106.801624
1,2024-01-01 21:00:00+07:00,NaN,NaN,NaN,-6.219151,106.801624
2,2024-01-02 20:00:00+07:00,NaN,NaN,NaN,-6.219151,106.801624
3,2024-01-02 21:00:00+07:00,NaN,NaN,NaN,-6.219151,106.801624
4,2024-01-03 20:00:00+07:00,NaN,NaN,NaN,-6.219151,106.801624
...,...,...,...,...,...,...
11699,2025-06-13 21:00:00+07:00,28.010500,2.902413,82.875084,-6.218538,106.801346
11700,2025-06-14 20:00:00+07:00,26.310501,6.569383,99.462250,-6.218538,106.801346
11701,2025-06-14 21:00:00+07:00,26.010500,4.693825,85.601379,-6.218538,106.801346
11702,2025-06-15 20:00:00+07:00,28.160500,2.189795,99.462250,-6.218538,106.801346


In [32]:
final_df.to_csv('wind_gbk_12_coordinate_one_year_v1.csv')